In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
import gzip
#import Recommender as Recommender
#import Evaluation as Evaluation

In [5]:
import dask as dd

In [25]:

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

dfmeta = getDF('/users/hp/desktop/Ds-springboard/capstone_2/data/meta_Electronics.json.gz')

In [26]:
dfmeta.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [27]:
dfmeta.shape

(498196, 9)

In [17]:
dfreviews=pd.read_csv('/users/hp/desktop/Ds-springboard/capstone_2/data/ratings_Electronics.csv', header=None,sep=',')



In [18]:
dfreviews.head()

,0,1,2,3
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [20]:
dfreviews.columns = ["reviewerID", "asin","overall","timestamp"]
dfreviews.head()

,reviewerID,asin,overall,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [21]:
dfreviews.shape

(7824482, 4)

In [8]:
#dfreviews.drop(['helpful', 'reviewText', 'summary','unixReviewTime','reviewTime'], axis=1, inplace=True)

In [28]:
product_ratings=pd.merge(dfreviews,dfmeta,on='asin',how='inner')

In [29]:
product_ratings.head()


,reviewerID,asin,overall,timestamp,imUrl,description,categories,title,price,salesRank,related,brand
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN


In [30]:
product_ratings.shape

(7824482, 12)

In [31]:
product_ratings.drop(['imUrl','description','categories','price','salesRank','related','brand'], axis=1, inplace=True)


In [33]:
product_ratings.drop(['timestamp'],axis=1,inplace=True)

In [47]:
product_ratings.head()

,reviewerID,asin,overall,title
0,AKM1MP6P0OYPR,0132793040,5.0,Kelby Training DVD: Mastering Blend Modes in A...
1,A2CX7LUOHB2NDG,0321732944,5.0,Kelby Training DVD: Adobe Photoshop CS5 Crash ...
2,A2NWSAGRHCP8N5,0439886341,1.0,Digital Organizer and Messenger
3,A2WNBOD3WNDNKT,0439886341,3.0,Digital Organizer and Messenger
4,A1GI0U4ZRJA8WN,0439886341,1.0,Digital Organizer and Messenger


In [48]:
product_ratings.shape

(7824482, 4)

In [49]:
product_ratings = product_ratings.rename(columns={'reviewerID': 'userID', 'asin': 'prod_ID','title': 'prod_name',
                                                 'overall': 'rating' })

In [50]:
product_ratings.head()

,userID,prod_ID,rating,prod_name
0,AKM1MP6P0OYPR,0132793040,5.0,Kelby Training DVD: Mastering Blend Modes in A...
1,A2CX7LUOHB2NDG,0321732944,5.0,Kelby Training DVD: Adobe Photoshop CS5 Crash ...
2,A2NWSAGRHCP8N5,0439886341,1.0,Digital Organizer and Messenger
3,A2WNBOD3WNDNKT,0439886341,3.0,Digital Organizer and Messenger
4,A1GI0U4ZRJA8WN,0439886341,1.0,Digital Organizer and Messenger


In [24]:
#product_ratings.to_csv('product_ratings.csv', sep='\t')

In [51]:
df=product_ratings

In [52]:
df.isnull().any().any()


True

In [53]:
df.isnull().sum().sum()


293557

In [54]:
df=df.dropna()


In [55]:
df.shape


(7530925, 4)

In [56]:
df.isnull().any().any()


False

In [58]:
len(df['userID'].unique())


4053964

In [59]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7530925 entries, 0 to 7824481
Data columns (total 4 columns):
userID       object
prod_ID      object
rating       float64
prod_name    object
dtypes: float64(1), object(3)
memory usage: 287.3+ MB


In [60]:
counts1=df['userID'].value_counts() 
counts=df['prod_ID'].value_counts()

In [61]:
counts1

A5JLAU2ARJ0BO     512
A3OXHLG6DIBRW8    482
ADLVFFE4VBT8      450
A6FIAB28IS79      424
A680RUE1FDO8B     395
A1ODOGXEYECQQ8    371
A2AY4YUOX2N1BQ    310
AWPODHOB4GFWL     302
A36K2N527TXXJN    291
ARBKYIVNYWK3C     288
A25C2M3QF9G7OQ    284
A22CW0ZHY3NJH8    281
A38RMU1Y5TDP9     278
A3EXWV8FNSSFL6    276
A3LGT6UZL99IW1    271
A2NOW4U7W3F7RI    269
A23GFTVIETX7DS    267
A3PD8JD9L4WEII    261
A231WM2Z2JL0U3    252
A12DQZKRKTNF5E    249
A3AYSYSLHU26U9    248
A2XRMQA6PJ5ZJ8    244
AGVWTYW0ULXHT     241
A203OCQQ12MAVT    240
A17BUUBOU0598B    239
A2NYK9KWFMJV4Y    238
A1UQBFCERIP7VJ    230
A3A4ZAIBQWKOZS    227
A1T1YSCDW0PD25    225
A250AXLRBVYKB4    224
                 ... 
AJBWQ5OPXACRZ       1
A2XD7AA1QR2WOV      1
A2GB0NHCVZ8I0E      1
AJ3YJTOAKFXVN       1
AP8IKHPFZ93MS       1
A3O750IJB8OW11      1
A262C85VA94EA3      1
A3BDEFTRI9TOZ2      1
A1LW7TIGZME5LF      1
A1VXY3UTQGYHJS      1
A24CXZEHZPJKVI      1
A20TWNQG8YIO89      1
A2ZRWR16VLG44       1
A1K4PDBVE5YS86      1
A3K2NCX5NU

In [62]:
counts

B0074BW614    18244
B00DR0PDNE    16454
B007WTAJTO    14172
B0019EHU8G    12285
B003ELYQGG    11617
B003ES5ZUU    10276
B00622AG6S     9823
B0002L5R78     9487
B008OHNZI0     8966
B003LR7ME6     8840
B000LRMS66     8715
B009SYZ8OC     8370
B004QK7HI8     7060
B009A5204K     7059
B00BGGDVOO     6893
B0098F5W0Q     6616
B002MAPRYU     6599
B002WE6D44     6509
B005HMKKH4     6134
B0012S4APK     5642
B0052YFYFK     5521
B0001FTVEK     5345
B0044YU60M     5239
B00316263Y     5038
B000I68BD4     4903
B006ZP8UOW     4842
B0041Q38NU     4774
B002V88HFE     4708
B0027VT6V4     4700
B000S5Q9CA     4690
              ...  
B003GIS8IY        1
B006K596GS        1
B00301U47A        1
B00005K2R1        1
B002NL3K6U        1
B007VJQ6UG        1
B00GWT7L6Y        1
B007HZLVTK        1
B0057Q0M1Y        1
B005DJYLS0        1
B002ON54PC        1
B001JTMX1A        1
B007JUU2V6        1
B00006JQ29        1
B002TTKCQW        1
B005CWT12O        1
B00BBZBRD2        1
B004AQJF8S        1
B00000K2GZ        1


In [63]:
len(counts1[counts1>=50].index)

1438

In [64]:
len(counts[counts>=50].index)


25406

In [65]:
df1=df[df['userID'].isin(counts1[counts1 >=50].index)]
df1.shape


(116794, 4)

In [66]:
df1=df1[df1['prod_ID'].isin(counts[counts >=50].index)]
df1.shape

(72341, 4)

In [67]:
df1.head()


,userID,prod_ID,rating,prod_name
492,A3CLWR1UUZT6TG,0972683275,5.0,VideoSecu 24&quot; Long Arm TV Wall Mount Low ...
631,A3TAS1AG6FMBQW,0972683275,5.0,VideoSecu 24&quot; Long Arm TV Wall Mount Low ...
1310,A3LDPF5FMB782Z,1400501466,5.0,"Barnes &amp; Noble NOOK Tablet 16gb (Color, BN..."
1322,A1A5KUIIIHFF4U,1400501466,1.0,"Barnes &amp; Noble NOOK Tablet 16gb (Color, BN..."
1335,A2XIOXRRYX0KZY,1400501466,3.0,"Barnes &amp; Noble NOOK Tablet 16gb (Color, BN..."


In [68]:
ratings_sum = pd.DataFrame(df1.groupby(['prod_ID'])['rating'].sum()).rename(columns = {'rating': 'ratings_sum'})
top10 = ratings_sum.sort_values('ratings_sum', ascending = False).head(10)
top10

,ratings_sum
prod_ID,
B003ES5ZUU,846.0
B0088CJT4U,814.0
B000N99BBC,755.0
B007WTAJTO,741.0
B00829TIEK,626.0
B00829THK0,560.0
B008DWCRQW,524.0
B004CLYEDC,517.0
B002R5AM7C,514.0


In [69]:
top10_popular=top10.merge(df1,left_index = True, right_on = 'prod_ID').drop_duplicates(
    ['prod_ID', 'prod_name'])[['prod_ID', 'prod_name','ratings_sum']]

In [70]:
print ('Top 10 Popular Products by sum user ratings\n')
top10_popular


Top 10 Popular Products by sum user ratings



,prod_ID,prod_name,ratings_sum
3313486,B003ES5ZUU,AmazonBasics High-Speed HDMI Cable - 15 Feet (...,846.0
6104981,B0088CJT4U,TP-LINK TL-WDR4300 Wireless N750 Dual Band Rou...,814.0
1198226,B000N99BBC,TP-LINK TL-SG1005D 10/100/1000Mbps 5-Port Giga...,755.0
5941380,B007WTAJTO,SanDisk Ultra 64GB MicroSDXC Class 10 UHS Memo...,741.0
6031403,B00829TIEK,Seagate Backup Plus 3TB USB 3.0 Desktop Extern...,626.0
6028195,B00829THK0,Seagate Backup Plus 1TB Desktop External Hard ...,560.0
6190152,B008DWCRQW,D-Link Wireless AC 1750 Mbps Home Cloud App-En...,524.0
4024382,B004CLYEDC,"Micra Digital CAT5e Snagless Patch Cable, 5 Fe...",517.0
2796338,B002R5AM7C,"Flip MinoHD Video Camera - Brushed Metal, 8 GB...",514.0
2883526,B002V88HFE,eneloop SEC-CSPACER4PK C Size Spacers for use ...,475.0


In [71]:
ratingsd=df1.pivot(index='prod_ID',columns= 'userID',values='rating').fillna(0)
ratingsd.head()

userID,A100UD67AHFODS,A100WO06OQR8BQ,A105S56ODHGJEK,A105TOJ6LTVMBG,A10AFVU66A79Y1,A10H24TDLK2VDP,A10NMELR4KX0J6,A10O7THJ2O20AG,A10PEXB6XAQ5XF,A10X9ME6R66JDX,...,AYOTEJ617O60K,AYP0YPLSP9ISM,AZ515FFZ7I2P7,AZ8XSDMIX04VJ,AZAC8O310IK4E,AZBXKUH4AIW3X,AZCE11PSTCH1L,AZMY6E8B52L2T,AZNUHQSHZHSUE,AZOK5STV85FBJ
prod_ID,,,,,,,,,,,,,,,,,,,,,
0972683275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1400501466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1400501520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1400501776,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1400532620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
ratingsd.shape


(16250, 1438)

In [73]:
from sklearn.model_selection import train_test_split
traind, testd = train_test_split(ratingsd, test_size=0.30,random_state=42)

In [74]:
train = traind.as_matrix()
test = testd.as_matrix()

/Users/hp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/Users/hp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [76]:
sparsity = float(len(train.nonzero()[0]))
sparsity /= (train.shape[0] * train.shape[1])
sparsity *= 100
print ('Sparsity: {:5.2f}%'.format(sparsity))


Sparsity:  0.31%


In [77]:
def item_similarity(ratings, epsilon=1e-9):
    # epsilon -> for handling dived-by-zero errors
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [78]:
item_sim = item_similarity(train)


In [79]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [80]:
item_prediction = predict_item(train, item_sim)


In [81]:
item_prediction[:4, :4]


array([[1.13154332e-02, 2.39846407e-13, 4.18406498e-13, 1.42814874e-02],
       [7.08353547e-13, 9.88597954e-03, 6.26177519e-03, 9.03053061e-04],
       [6.14677728e-02, 5.25546476e-02, 6.00339326e-02, 3.81653351e-02],
       [3.07085918e-13, 4.71910825e-03, 2.83874998e-13, 3.87580711e-13]])

In [82]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [84]:
print ('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))


Item-based CF MSE: 19.474033380362258


In [85]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(traind) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pm=pd.DataFrame(cosine_similarity(r_mat_tr))
#pm.head()
ctrain = cosine_similarity(r_mat_tr)


[0.0146434  0.00719657 0.00617064 0.0047372  0.00449876 0.00414031
 0.00404674 0.00397643 0.00385144 0.0036875  0.00363512 0.0034617
 0.00339788 0.00330323 0.00321327 0.00316889 0.00310721 0.00307747
 0.00299119 0.00296976 0.00287676 0.00285001 0.00279331 0.00275791
 0.00275023 0.00272965 0.00269243 0.00268028 0.00264847 0.00263828
 0.00260773 0.00254477 0.00251658 0.00250013 0.00247933 0.0024676
 0.00241899 0.00239642 0.0023941  0.00237252 0.00233962 0.00234226
 0.00228751 0.00228433 0.00226599 0.00220661 0.00219082 0.00216018
 0.0021291  0.00209723]
0.16369386193637217


In [86]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(testd) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pmtt=pd.DataFrame(cosine_similarity(r_mat_tr))
#print (pmtt[:2])
#pmtt.head()
ctest = cosine_similarity(r_mat_tr)


[0.02010026 0.00928553 0.00665454 0.0061458  0.00586076 0.00558876
 0.00524494 0.00512105 0.00502567 0.00495611 0.00477533 0.00459343
 0.00454924 0.00431174 0.00421176 0.00415563 0.00409069 0.00398067
 0.00389391 0.00385626 0.00376387 0.00370799 0.00366667 0.00357465
 0.00350927 0.00343089 0.0033947  0.00334115 0.00329635 0.00322994
 0.00320732 0.00316657 0.00315119 0.00313284 0.00305854 0.00304108
 0.00301784 0.00299211 0.00292439 0.00290545 0.00287098 0.00284074
 0.00282324 0.00280391 0.00279295 0.002764   0.00275652 0.00269294
 0.0026512  0.00264159]
0.20955292096154493


In [87]:
print (' CF MSE: ' + str(get_mse(ctrain, ctest)))


 CF MSE: 0.040622279780595204


In [88]:
df1 = df1.sort_values(by='rating')
df1 = df1.reset_index(drop=True)
count_users = df1.groupby("userID", as_index=False).count()


In [89]:
count = df1.groupby("prod_ID", as_index=False).mean()


In [90]:
items_df = count[['prod_ID']]
items_df.head()
print(len(items_df))


16250


In [91]:
users_df = count_users[['userID']]
users_df.head()
print(len(users_df))


1438


In [92]:
users_list = users_df.values
len(users_list)

1438

In [93]:
df_clean_matrix = df1.pivot(index='prod_ID', columns='userID', values='rating').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).as_matrix()
R


/Users/hp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [94]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.02418462, -0.02418462, -0.02418462, ..., -0.02418462,
        -0.02418462, -0.02418462],
       [-0.02024615, -0.02024615, -0.02024615, ..., -0.02024615,
        -0.02024615, -0.02024615],
       [-0.00861538, -0.00861538, -0.00861538, ..., -0.00861538,
        -0.00861538, -0.00861538],
       ...,
       [-0.02861538, -0.02861538, -0.02861538, ..., -0.02861538,
        -0.02861538, -0.02861538],
       [-0.00793846, -0.00793846, -0.00793846, ..., -0.00793846,
        -0.00793846, -0.00793846],
       [-0.00886154, -0.00886154, -0.00886154, ..., -0.00886154,
        -0.00886154, -0.00886154]])

In [95]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned)

In [96]:
sigma = np.diag(sigma)


In [97]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_clean_matrix.columns)
preds_df['userID'] = users_df
preds_df.set_index('userID', inplace=True)
preds_df.head()

prod_ID,0972683275,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,1400599997,1400698987,3744295508,...,B00JTI4X3E,B00JX1ZS5O,B00K0OBEE2,B00K4VQZCM,B00K7O2DJU,B00K91DB7Y,B00KFAGCUM,B00KINSDK8,B00KWHMR6G,B00L3YHF6O
userID,,,,,,,,,,,,,,,,,,,,,
A100UD67AHFODS,0.015423,0.011009,0.013081,0.016089,0.011947,0.018877,0.012717,0.007655,0.013124,0.011771,...,0.009765,0.393998,0.034516,0.050864,0.013123,0.045171,0.000450,0.061026,0.008367,0.362923
A100WO06OQR8BQ,0.024796,0.006530,-0.001214,0.012578,-0.000154,0.005729,-0.001113,0.007468,0.003846,-0.001871,...,-0.001331,0.077428,-0.002063,0.015925,0.021663,0.244201,0.078147,-0.006486,0.006921,0.172743
A105S56ODHGJEK,-0.003408,0.001779,-0.002836,0.002212,-0.000092,-0.006465,-0.002632,0.008081,-0.001386,-0.002772,...,-0.001007,-0.043161,0.006384,0.015014,0.015221,0.224464,0.063182,0.003796,0.003310,0.024768
A105TOJ6LTVMBG,0.015205,0.008016,0.007105,0.008773,0.004356,0.012288,0.006753,0.004501,0.010781,0.006014,...,0.004625,0.005234,-0.003433,0.005988,0.008871,-0.017516,0.014102,-0.003865,0.006301,0.023960
A10AFVU66A79Y1,0.009980,0.006588,0.002785,0.009932,0.003707,0.004818,0.002853,0.008817,0.003796,0.002749,...,0.003077,-0.007986,0.001601,0.005722,0.013716,0.123540,0.032900,0.000953,0.007231,0.045863


In [98]:
preds_df.shape


(1438, 16250)

In [99]:
def recommend_it(predictions_df, itm_df, original_ratings_df, num_recommendations=10,ruserId='A100UD67AHFODS'):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[ruserId].sort_values(ascending=False)
    
    # Get the user's data and merge in the item information.
    user_data = original_ratings_df[original_ratings_df.userID == ruserId]
    user_full = (user_data.merge(itm_df, how = 'left', left_on = 'prod_ID', right_on = 'prod_ID').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already purchased {1} items.'.format(ruserId, user_full.shape[0]))
    print ('Recommending the highest {0} predicted  items not already purchased.'.format(num_recommendations))
    
    # Recommend the highest predicted rating items that the user hasn't bought yet.
    recommendations = (itm_df[~itm_df['prod_ID'].isin(user_full['prod_ID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'prod_ID',
               right_on = 'prod_ID').
         rename(columns = {ruserId: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    topk=recommendations.merge(original_ratings_df,left_index = True, right_on = 'prod_ID',left_on='prod_ID').drop_duplicates(
    ['prod_ID', 'prod_name'])[['prod_ID', 'prod_name']]

    return topk

In [100]:
recommend_it(preds_df, items_df, df1, 5)


User A100UD67AHFODS has already purchased 85 items.
Recommending the highest 5 predicted  items not already purchased.


,prod_ID,prod_name
29131,B00G4UQ6U8,Brainwavz S1 In Ear Headphones
7834,B007OY5V68,NEW AYL&reg; Portable Mini Speaker System with...
10755,B00FSA8VQ2,Brainwavz Delta IEM Earphones
3596,B003ES5ZUU,AmazonBasics High-Speed HDMI Cable - 15 Feet (...
25798,B00DTZYHX4,Bolse&reg; 300Mbps Wifi Wireless-N USB Micro M...


In [101]:
#recommend for any user
recommend_it(preds_df, items_df, df1, 5,'A100WO06OQR8BQ')


User A100WO06OQR8BQ has already purchased 91 items.
Recommending the highest 5 predicted  items not already purchased.


,prod_ID,prod_name
11745,B004CLYEDC,"Micra Digital CAT5e Snagless Patch Cable, 5 Fe..."
5649,B000N99BBC,TP-LINK TL-SG1005D 10/100/1000Mbps 5-Port Giga...
5012,B004CLYEFK,Micra Digital USB A to USB B Cable (6 Feet)
1169,B00829THK0,Seagate Backup Plus 1TB Desktop External Hard ...
656,B00834SJSK,Seagate Expansion 500GB Portable External Hard...
